使用 FastAPI，你可以定义、校验、记录文档并使用任意深度嵌套的模型（归功于Pydantic）。

## 嵌套模型

Pydantic 模型的每个属性都具有类型。

但是这个类型本身可以是另一个 Pydantic 模型。

因此，你可以声明拥有特定属性名称、类型和校验的深度嵌套的 JSON 对象。

上述这些都可以

例如，我们可以定义一个 Image 模型

然后我们可以将其用作一个属性的类型：任意的嵌套。法。st：



In [1]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()


class Image(BaseModel):
    url: str
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    image: Image | None = None


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:59819 - "PUT /items/5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23660]


这意味着 FastAPI 将期望类似于以下内容的请求体：
{
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": ar"],
    "image": {
        "url": "http://example.com/baz.jpg",
        "name": "The Foo live"

仅仅进行这样的声明，你将通过 FastAPI 获得：

对被嵌入的模型也适用的编辑器支持（自动补全等）
数据转换
数据校验
自动生成文档
    }
}

## requests库

In [ ]:
url = 'http://127.0.0.1:8009/items/5' 
data = {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "image": {
        "url": "http://example.com/baz.jpg",
        "name": "The Foo live"
    }
}
res = requests.put(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 
# '{"item_id":5,"item":{"name":"Foo","description":"The pretender","price":42.0,"tax":3.2,"image":{"url":"http://example.com/baz.jpg","name":"The Foo live"}}}'

## 特殊的类型和校验

除了普通的单一值类型（如 str、int、float 等）外，你还可以使用从 str 继承的更复杂的单一值类型。

例如，在 Image 模型中我们有一个 url 字段，我们可以把它声明为 Pydantic 的 HttpUrl，而不是 str：

该字符串将被检查是否为有效的 URL，并在 JSON Schema / OpenAPI 文档中进行记录。

In [ ]:
from pydantic import BaseModel, HttpUrl
class Image(BaseModel):
    url: HttpUrl
    name: str

## 带有一组子模型的属性

In [ ]:
你还可以将 Pydantic 模型用作 list、set 等的子类型：

In [2]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, HttpUrl

app = FastAPI()


class Image(BaseModel):
    url: HttpUrl
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    images: list[Image] | None = None


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    results = {"item_id": item_id, "item": item}
    return results

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:59913 - "PUT /items/5 HTTP/1.1" 200 OK
INFO:     127.0.0.1:59999 - "PUT /items/5 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23660]


这将期望（转换，校验，记录文档等）下面这样的 JSON 请求体：
{
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": "
    ],
    "images": [
        {
            "url": "http://example.com/baz.jpg",
            "name": "The Foo live"
        },
        {
            "url": "http://example.com/dave.jpg",
            "name": "The Baz"
        }
    ]
}

In [ ]:
url = 'http://127.0.0.1:8009/items/5' 
data = {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "images": [
        {
            "url": "http://example.com/baz.jpg",
            "name": "The Foo live"
        },
        {
            "url": "http://example.com/dave.jpg",
            "name": "The Baz"
        }
    ]
}
res = requests.put(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 
# '{"item_id":5,"item":{"name":"Foo","description":"The '
#  'pretender","price":42.0,"tax":3.2,"images":[{"url":"http://example.com/baz.jpg","name":"The '
#  'Foo live"},{"url":"http://example.com/dave.jpg","name":"The Baz"}]}}'

## 深度嵌套模型

你可以定义任意深度的嵌套模型：

In [3]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, HttpUrl

app = FastAPI()


class Image(BaseModel):
    url: HttpUrl
    name: str


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None
    images: list[Image] | None = None

class Offer(BaseModel):
    name: str
    description: str | None = None
    price: float
    items: list[Item]


@app.post("/offers/")
async def create_offer(offer: Offer):
    return offer

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:60226 - "POST /offers/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23660]


In [ ]:
这将期望（转换，校验，记录文档等）下面这样的 JSON 请求体：
{
    "name": "FooOffer",
    "description": "The pretender Offer",
    "price": 421.0,
    "items": [
        {
            "name": "Foo1",
            "description": "The pretender1",
            "price": 42.0,
            "tax": 3.2,
            "images": [
                {
                    "url": "http://example.com/baz1.jpg",
                    "name": "The Foo live1"
                },
                {
                    "url": "http://example.com/dave1.jpg",
                    "name": "The Baz1"
                }
            ]
        },
        {
            "name": "Foo2",
            "description": "The pretender2",
            "price": 43.0,
            "tax": 3.3,
            "images": [
                {
                    "url": "http://example.com/baz2.jpg",
                    "name": "The Foo live2"
                },
                {
                    "url": "http://example.com/dave2.jpg",
                    "name": "The Baz2"
                }
            ]
        }
    ]
}


## requests库

In [ ]:
url = 'http://127.0.0.1:8009/offers/' 
data = {
    "name": "FooOffer",
    "description": "The pretender Offer",
    "price": 421.0,
    "items": [
        {
            "name": "Foo1",
            "description": "The pretender1",
            "price": 42.0,
            "tax": 3.2,
            "images": [
                {
                    "url": "http://example.com/baz1.jpg",
                    "name": "The Foo live1"
                },
                {
                    "url": "http://example.com/dave1.jpg",
                    "name": "The Baz1"
                }
            ]
        },
        {
            "name": "Foo2",
            "description": "The pretender2",
            "price": 43.0,
            "tax": 3.3,
            "images": [
                {
                    "url": "http://example.com/baz2.jpg",
                    "name": "The Foo live2"
                },
                {
                    "url": "http://example.com/dave2.jpg",
                    "name": "The Baz2"
                }
            ]
        }
    ]
}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 

# '{"name":"FooOffer","description":"The pretender Offer","price":421.0,"items":[{"name":"Foo1","description":"The pretender1","price":42.0,"tax":3.2,"images":[{"url":"http://example.com/baz1.jpg","name":"The Foo live1"},{"url":"http://example.com/dave1.jpg","name":"The Baz1"}]},{"name":"Foo2","description":"The pretender2","price":43.0,"tax":3.3,"images":[{"url":"http://example.com/baz2.jpg","name":"The Foo live2"},{"url":"http://example.com/dave2.jpg","name":"The Baz2"}]}]}'

## 纯列表请求体

如果你期望的 JSON 请求体的最外层是一个 JSON array（即 Python list），

则可以在路径操作函数的参数中声明此类型，就像声明 Pydantic 模型一样：

images: List[Image]

In [4]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, HttpUrl

app = FastAPI()


class Image(BaseModel):
    url: HttpUrl
    name: str


@app.post("/images/multiple/")
async def create_multiple_images(images: list[Image]):
    return images

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:60298 - "POST /images/multiple/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23660]


这将期望（转换，校验，记录文档等）下面这样的 JSON 请求体：
[
    {
        "url": "http://example.com/baz.jpg",
        "name": "The Foo live"
    },
    {
        "url": "http://example.com/dave.jpg",
        "name": "The Baz"
    }
]

## requests库

In [ ]:
url = 'http://127.0.0.1:8009/images/multiple/' 
data = [
    {
        "url": "http://example.com/baz.jpg",
        "name": "The Foo live"
    },
    {
        "url": "http://example.com/dave.jpg",
        "name": "The Baz"
    }
]
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 
# '[{"url":"http://example.com/baz.jpg","name":"The Foo live"},{"url":"http://example.com/dave.jpg","name":"The Baz"}]'

## 任意 dict 构成的请求体

你也可以将请求体声明为使用某类型的键和其他类型值的 dict。

无需事先知道有效的字段/属性（在使用 Pydantic 模型的场景）名称是什么。

如果你想接收一些尚且未知的键，这将很有用。

In [5]:
import uvicorn
from fastapi import FastAPI

app = FastAPI()


@app.post("/index-weights/")
async def create_index_weights(weights: dict):
    return weights

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [23660]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:60372 - "POST /index-weights/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:60389 - "POST /index-weights/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23660]


在这种情况下，只要传进来的数据最外层是个dict类型的数据即可。不用管里面的内容。

In [ ]:
url = 'http://127.0.0.1:8009/index-weights/' 
data = {"item_id": 5, "name": "张三"}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"item_id":5,"name":"张三"}'

字典里面嵌套列表也没问题：

In [ ]:
url = 'http://127.0.0.1:8009/index-weights/' 
data = {
    "name": "Foo",
    "description": "The pretender",
    "price": 42.0,
    "tax": 3.2,
    "images": [
        {
            "url": "http://example.com/baz.jpg",
            "name": "The Foo live"
        },
        {
            "url": "http://example.com/dave.jpg",
            "name": "The Baz"
        }
    ]
}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到
# '{"name":"Foo","description":"The pretender","price":42.0,"tax":3.2,"images":[{"url":"http://example.com/baz.jpg","name":"The Foo live"},{"url":"http://example.com/dave.jpg","name":"The Baz"}]}'